# Load Module and data

In [2]:
from posixpath import join
from astropy.io.fits import hdu
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
from ciao_contrib.runtool import search_csc 

from astropy.io import fits 
from fields import fields 
from tqdm import tqdm

In [26]:
df = pd.read_csv('../source_list/atnf_catalogue_full.csv' , delimiter=';' , comment='#' , na_values='*' , index_col='INDEX')
df = df.replace(' ' , '_' ,regex=True)
assoc = df['ASSOC'].replace(np.nan , 'NA:')
assoc = [s.split(':')[0] for s in assoc]
len(assoc)
df.insert(7 , 'ASSOC_MOD' , assoc)
df 

,NAME,PSRJ,RAJ,DECJ,RAJD,DECJD,BINCOMP,ASSOC_MOD,ASSOC,SURVEY,PSR,Unnamed: 11
INDEX,,,,,,,,,,,,
1,J0002+6216,J0002+6216,00:02:58.1,+62:16:09.4,0.742375,62.269278,NaN,GRS,"GRS:4FGL_J0002.8+6217[fermi21],XRS:1XSPS",FermiBlind,HE[wcp+18],NaN
2,J0006+1834,J0006+1834,00:06:04.8,+18:34:59,1.520000,18.583056,NaN,NA,NaN,ar4,NaN,NaN
3,J0007+7303,J0007+7303,00:07:01.7,+73:03:07.4,1.757083,73.052056,NaN,GRS,"GRS:4FGL_J0007.0+7303[fermi21],XRS:RX_J0",FermiBlind,NRAD,NaN
4,J0011+08,J0011+08,00:11:34,+08:10,2.891667,8.166667,NaN,NA,NaN,ar327,NaN,NaN
5,B0011+47,J0014+4746,00:14:17.7,+47:46:33.4,3.573958,47.775944,NaN,NA,NaN,"gb1,gb2,gb3,lotaas",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3173,J2351+8533,J2351+8533,23:51:03,+85:33:20.6,357.763734,85.555735,NaN,NA,NaN,"gbncc,lotaas",NaN,NaN
3174,J2352+65,J2352+65,23:52,+65,358.000000,65.000000,NaN,NA,NaN,"gb350,gbncc,lotaas",NaN,NaN
3175,J2354-22,J2354-22,23:54:26,-22:51:53,358.608333,-22.864722,NaN,NA,NaN,htru_pks,NaN,NaN


# Data Analysis

In [28]:
num_src = len(df)
num_unclf_src = len(df[df['PSR'].isna()])
num_clf_src=  num_src - num_unclf_src 
print('Totoal Number of sources : ' , num_src)
print('Unclassified sources : ' , num_unclf_src)
print('Total classified sources : ' ,  num_clf_src)
df['ASSOC_MOD'].value_counts().to_frame()

Totoal Number of sources :  3177
Unclassified sources :  2726
Total classified sources :  451


,ASSOC_MOD
NA,2615
GRS,293
GC,152
XRS,48
EXGAL,30
SNR,27
OPT,11
SGR_J1745-29[kbk+13],1


# Download data

In [30]:
start = int(input('Start number : '))
end = int(input('End number : '))

#start = 0 
#end = 3

df_select = df.iloc[start:end].reset_index(drop=True)

ra = df_select['RAJ']
dec = df_select['DECJ']
name = df_select['NAME']
num_obs = [] 
offset = []
for index in tqdm(range(len(df_select))):
    try :
        s = search_csc(
                        radunit="arcsec", 
                        columns=fields, 
                        bands="broad",
                        clobber="yes" ,
                        radius= 15 , 
                        pos= str(ra[index])+','+str(dec[index])  ,
                        outfile='temp.csv')
                    #sys('search_csc outfile=trial.csv radunit=arcsec columns="SOS,SOP,SOV , o.gti_obs m.flux_aper_b" bands=broad clobber=yes radius=1 pos="65.428058,32.907468"')
                    #print(s)
                    
        data = pd.read_csv('temp.csv', delimiter='\t' , comment='#')
        data = data[data['match_type']=='u          ']
        data = data[data['instrument']=='ACIS']
        data.index.name= 'index'
        #f_name_fits = data_fits['A_NAME']
        n = len(data)
        num_obs.append(n)
        if(n>0):
            print('Match found --' , len(data))
            data.to_csv('../data/all_data/PULSAR_v2_all/'+name[index]+'.csv')
    except:
        print('Manual ispection needed :' , name[index])
        num_obs.append(-1)

100%|██████████| 3/3 [00:08<00:00,  2.76s/it]

Match found -- 3


In [32]:
df_updated = df_select.copy()
df_updated.insert(5 , 'num_obs' , num_obs)

df_updated.to_csv('../source_list/pulsar_updated/'+str(start)+'-'+str(end)+'.csv')